<a href="https://colab.research.google.com/github/Ashish-Tripathy/cv_yantra_drishti/blob/master/LSTM_Final_Generate_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation using LSTM
In this following submission, I have showcased how we can replace the solution from https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ to make our generator better.
We have done the following:
1. Introduced padded sequences by first using a separator to separate the text into arrays, generating input sequences out of each array element and then padding the input sequences to a decided length
2. We have generated 500 sequence of characters from our models, though our models were best predicting shorter character sequences much better due to the way we generate input sequences
3. Trained the model for 500 epochs
4. Added dropout of 0.1 for each lstm layer
5. Explored multiple separators
6. Explored word based text genaration
  

In [0]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.backend import expand_dims


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Reading the data

In [0]:
# load ascii text and covert to lowercase
filename = "/content/drive/My Drive/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


# Character based Models

## Using '.' as separator

In [0]:
# to create variable sequence of sentences, i will remove puntctuations except '.' which will be our separator
import string
for x in raw_text: 
  remove = string.punctuation
  remove = remove.replace(".", "")
  if x in remove:
    raw_text = raw_text.replace(x, "") 

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [0]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


Total Characters:  136958
Total Vocab:  29


For each sequence generate sequences using per character, for eg. if we have a sequence generated by separating using '.' -  alice in the wonderland.
For this sequence we will generate input sequence like:

a

al

ali

alic

alice

alice 

alice i

alice in

alice in 

alice in t

alice in th

alice in the

alice in the 

alice in the w....... like this until 'wonderland'

In [0]:
#separate by '.'
sequence = raw_text.split('.')
#generate input sequence
input_sequences = []
for line in sequence:
    char_encodings = ([char_to_int[char] for char in line])
    for i in range(1, len(char_encodings)):
      input_sequences.append(char_encodings[:i+1])
      

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,   
                      maxlen=100, padding='pre'))

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])

In [0]:
input_sequences

array([[ 0,  0,  0, ...,  0,  3, 14],
       [ 0,  0,  0, ...,  3, 14, 11],
       [ 0,  0,  0, ..., 14, 11,  5],
       ...,
       [21,  1,  3, ...,  1,  6,  3],
       [ 1,  3, 16, ...,  6,  3, 27],
       [ 3, 16,  6, ...,  3, 27, 21]], dtype=int32)

### Training data prapration

for the same 'alice in wodeland'

X = a; y = l

X = al; y = i

X = ali; y = c

and so on...



In [0]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = np_utils.to_categorical(label)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(predictors, (len(predictors), max_sequence_len-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = label

### Building Model architecture

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.1, return_sequences=True))
model.add(LSTM(256,dropout=0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0725 06:46:26.123198 140156570359680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 06:46:26.164747 140156570359680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 06:46:26.176585 140156570359680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 06:46:26.419962 140156570359680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0725 06:46:26.431972 

### Training the model

In [0]:
filepath="/content/drive/My Drive/Colab Notebooks/EIPModels/EIP2/textGenerationWFS-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)

Epoch 1/100
134982/134982 [==============================] - 82s 606us/step - loss: 2.8710

Epoch 00001: loss improved from inf to 2.87101, saving model to /content/drive/My Drive/Colab Notebooks/EIPModels/EIP2/textGenerationWFS-01-2.8710-bigger.hdf5
Epoch 2/100
134982/134982 [==============================] - 79s 586us/step - loss: 2.7197

Epoch 00002: loss improved from 2.87101 to 2.71972, saving model to /content/drive/My Drive/Colab Notebooks/EIPModels/EIP2/textGenerationWFS-02-2.7197-bigger.hdf5
Epoch 3/100
134982/134982 [==============================] - 79s 585us/step - loss: 2.5049

Epoch 00003: loss improved from 2.71972 to 2.50494, saving model to /content/drive/My Drive/Colab Notebooks/EIPModels/EIP2/textGenerationWFS-03-2.5049-bigger.hdf5
Epoch 4/100
134982/134982 [==============================] - 80s 593us/step - loss: 2.3224

Epoch 00004: loss improved from 2.50494 to 2.32241, saving model to /content/drive/My Drive/Colab Notebooks/EIPModels/EIP2/textGenerationWFS-04-2.3

In [0]:
pattern.shape

(73,)

In [0]:
# pick a random seed
start = numpy.random.randint(0, len(input_sequences)-1)
pattern = input_sequences[start][:-1]
print ("Seed:")
print ( (''.join([int_to_char[value] for value in pattern])).lstrip())

for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  #result += " "+result
  sys.stdout.write(result)
  pattern = np.append(pattern,index)
  pattern = pattern[1:len(pattern)]

print ("\nDone.")

Seed:
they cant have anything to put
down yet before the trials be
out hor the hanl asd tuleby mtsk
at if hauhen it a large caner sdtsing sain the had
nov a lowt of taid the had
notw homes anice colcued wenl tomeiny sarked youd bnd atwinusly reparkasd abtkiloed atas lurglyed at the san anice comping un the
wery dowid in rntrer suestion anlce and ie walk woure tatting about forught toeep and hlgsgit the whougst whers tuch a crpk to the
whough you   little before so the wail wo her fanl a darcupill mabel turning eurwtiyer the when of tersehty inm hit tostling abo
Done.


### Genarated text:

seed: 

they cant have anything to put
down yet before the trials be

Text generated: 

out hor the hanl asd tuleby mtsk
at if hauhen it a large caner sdtsing sain the had
nov a lowt of taid the had
notw homes anice colcued wenl tomeiny sarked youd bnd atwinusly reparkasd abtkiloed atas lurglyed at the san anice comping un the
wery dowid in rntrer suestion anlce and ie walk woure tatting about forught toeep and hlgsgit the whougst whers tuch a crpk to the
whough you   little before so the wail wo her fanl a darcupill mabel turning eurwtiyer the when of tersehty inm hit tostling abo

## Using 30th space as separator

In [0]:
import string
for x in raw_text: 
    if x in string.punctuation:
        raw_text = raw_text.replace(x, "") 

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [0]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


Total Characters:  135970
Total Vocab:  28


In [0]:
span = 30
sequence = raw_text.split(' ')
sequence = [" ".join(sequence[i:i+span]) for i in range(0, len(sequence), span)] #splitting on every 30th space

In [0]:
input_sequences = []
for line in sequence:
    char_encodings = ([char_to_int[char] for char in line])#tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(char_encodings)):
      input_sequences.append(char_encodings[:i+1])
      

In [0]:
input_seq = np.array(pad_sequences(input_sequences,maxlen=100, padding='pre'))
max_sequence_len = max([len(x) for x in input_seq])

In [0]:
predictors, label = input_seq[:,:-1],input_seq[:,-1]
label = np_utils.to_categorical(label)

In [0]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(predictors, (len(predictors), max_sequence_len-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = label

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.1, return_sequences=True))
model.add(LSTM(256,dropout=0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath="/content/drive/My Drive/EIP2/Best models/textGeneration-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)

Epoch 1/100
134301/134301 [==============================] - 79s 586us/step - loss: 2.8772

Epoch 00001: loss improved from inf to 2.87725, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-01-2.8772-bigger.hdf5
Epoch 2/100
134301/134301 [==============================] - 77s 575us/step - loss: 2.7263

Epoch 00002: loss improved from 2.87725 to 2.72626, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-02-2.7263-bigger.hdf5
Epoch 3/100
134301/134301 [==============================] - 78s 577us/step - loss: 2.5175

Epoch 00003: loss improved from 2.72626 to 2.51750, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-03-2.5175-bigger.hdf5
Epoch 4/100
134301/134301 [==============================] - 78s 581us/step - loss: 2.3335

Epoch 00004: loss improved from 2.51750 to 2.33353, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-04-2.3335-bigger.hdf5
Epoch 5/100
134301/134301 [=========================

In [0]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/EIP2/Best models/textGeneration-100-0.6618-bigger.hdf5')

W0727 14:27:09.339074 140701900285824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 14:27:09.380603 140701900285824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 14:27:09.388442 140701900285824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 14:27:09.655366 140701900285824 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0727 14:27:09.671676 

In [0]:
# pick a random seed
start = numpy.random.randint(0, len(input_sequences)-1)
pattern = predictors[start]
print ("Seed:")
print ( (''.join([int_to_char[value] for value in pattern])).lstrip())

for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  #result += " "+result
  sys.stdout.write(result)
  pattern = np.append(pattern,index)
  pattern = pattern[1:len(pattern)]

print ("\nDone.")


Seed:
ly round to see if he would deny it too but the dormouse denied
nothing being fast asleep

after th
ared like to begin anl ever toe couter of alice as hen footistoaus themr what toeak tuupid
alice tuiet tulemer
 alice could herself in mine tur inme toeakne suisented the hedlery what ialittle hirher what ie yotklng oeaus youn nakes the waid to lert each nine tou thenish po toeep oitsle uipink forseruueset hir thadh at hol anl thes tureard doos becr the had nodeed sathahe

whe slal sattuinner

alice wail oe for
hirst hererally as iatolams respoe wery toow lnowtession what iadhent whine soeedhng 
Done.


<b>Seed:</b>
ly round to see if he would deny it too but the dormouse denied
nothing being fast asleep


<b>Generated text: </b>
after th
ared like to begin anl ever toe couter of alice as hen footistoaus themr what toeak tuupid
alice tuiet tulemer
 alice could herself in mine tur inme toeakne suisented the hedlery what ialittle hirher what ie yotklng oeaus youn nakes the waid to lert each nine tou thenish po toeep oitsle uipink forseruueset hir thadh at hol anl thes tureard doos becr the had nodeed sathahe

whe slal sattuinner

alice wail oe for
hirst hererally as iatolams respoe wery toow lnowtession what iadhent whine soeedhng


## Other creative separators explored:

1. random nth space to generate input sequences like the fixed sequence method used in machine learning mastery
2. '\n' as separator

# Using Word based Generators

Explored Word based generators which are widely used for text generation as it creates a dictionary for each word to be used, so the geneated text atleast has correct words, the model has to learn in what context to use the words.

In [0]:
import string
for x in raw_text: 
    if x in string.punctuation:
        raw_text = raw_text.replace(x, "") 

## Using '\n' as separator

Building \n as separator, fit a tokenizer() which forms tokens for each word in the text

In [0]:
tokenizer = Tokenizer()
raw_text = raw_text.split("\n")
tokenizer.fit_on_texts(raw_text)
total_words = len(tokenizer.word_index) + 1

## Create input sequences 
Text: Alice in Wonderland

Alice

Alice in

Alice in Wonderland



In [0]:
token_list = tokenizer.texts_to_sequences(raw_text)[0]
for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

## Padding the sequences with 0s to create fixed length sequences

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,   
                      maxlen=max_sequence_len, padding='pre'))

## Training data preparation

In [0]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = np_utils.to_categorical(label, num_classes=total_words)

## Building the model architecture

In [0]:
input_len = max_sequence_len - 1
model = Sequential()

# Add Input Embedding Layer
model.add(Embedding(total_words, 10, input_length=input_len))
# adding just one layer of lstm to test the approach
model.add(LSTM(256, dropout = 0.1))
#Add Output Layer
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


## Training the model

In [0]:
filepath="/content/drive/My Drive/EIP2/Best models/textGeneration-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(predictors, label, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
23904/23904 [==============================] - 5s 197us/step - loss: 6.4370

Epoch 00001: loss improved from inf to 6.43696, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-01-6.4370-bigger.hdf5
Epoch 2/100
23904/23904 [==============================] - 3s 123us/step - loss: 6.0470

Epoch 00002: loss improved from 6.43696 to 6.04704, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-02-6.0470-bigger.hdf5
Epoch 3/100
23904/23904 [==============================] - 3s 119us/step - loss: 5.9630

Epoch 00003: loss improved from 6.04704 to 5.96295, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-03-5.9630-bigger.hdf5
Epoch 4/100
23904/23904 [==============================] - 3s 119us/step - loss: 5.8882

Epoch 00004: loss improved from 5.96295 to 5.88816, saving model to /content/drive/My Drive/EIP2/Best models/textGeneration-04-5.8882-bigger.hdf5
Epoch 5/100
23904/23904 [==============================] - 3s 12

## Generating the text

Generating 500 words from any seed chosen at random from the text

In [0]:
seed_text = "making faces at him as he spoke"
for _ in range(500):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model.predict_classes(token_list, verbose=0)

  output_word = ""
  for word,index in tokenizer.word_index.items():
      if index == predicted:
          output_word = word
          break
  seed_text += " "+output_word
print(seed_text.title().lower())

making faces at him as he spoke to get in the clock in a well air it was talking the list of the deepest as the rest of the table as it was his feet high he felt all the list of expecting to begin with her to settle as she spoke and the moral of the song the king was leaning with the wood and the white rabbit blew three blasts on the trumpet and then he would be the right thing said the mock turtle washingextra disagree with a great hurry and the fan and was suppressed under the little drawlingthe key and unlocking the door and the pair of the gloves the king went on the gloves and the gloves he was speaking to rightly as she was beginning to rightly sure i am i think to be sure i am to be otherwise messages wondering but the officer could get in a court sometimes choked with the pope civil in a minute or kettle was all the list of the deepest as the moment the king added swim the gloves she had caught the hatter and the hatter hurriedly left in the sun and had just in the same head as

<b>seed</b>:making faces at him as he spoke


<b>Generated text</b>: to get in the clock in a well air it was talking the list of the deepest as the rest of the table as it was his feet high he felt all the list of expecting to begin with her to settle as she spoke and the moral of the song the king was leaning with the wood and the white rabbit blew three blasts on the trumpet and then he would be the right thing said the mock turtle washingextra disagree with a great hurry and the fan and was suppressed under the little drawlingthe key and unlocking the door and the pair of the gloves the king went on the gloves and the gloves he was speaking to rightly as she was beginning to rightly sure i am i think to be sure i am to be otherwise messages wondering but the officer could get in a court sometimes choked with the pope civil in a minute or kettle was all the list of the deepest as the moment the king added swim the gloves she had caught the hatter and the hatter hurriedly left in the sun and had just in the same head as he fumbled over the list of the creature and the queen was busily up and the moment he was gone and it was over to get in a melancholy livery had just just just at the hatter i mean what a nice little thing howled along the gryphon and the hatter was over with a great hurry in the wood and the pair of the suppressed guineapigs instantly and all the queen was in the direction the king added swim in a candle she had caught the flamingo and he poured a little wider about a trumpet and then she had not to talk in any rate she had never to sing you join the dance and in a minute or two she walked up and the dormouse was over with a great hurry in sneezes energetic it was beginning won and she tried to curtsey in a few minutes which seemed to be full of soup and dry leaves to be sure i am i think to be otherwise among the glass table and i have what to lie down on it and she had not to me so i cant get in a natural livery came trotting along the queen just in a pie was a real tale and the hatter was over with a great hurry in the wood and the pair of the suppressed guineapigs instantly and all the queen was in the direction the king added swim in a candle she had caught the flamingo and he poured a little wider about a trumpet and then she had not to talk in any rate she had never to sing you join the dance and in a minute or two she walked up and the dormouse was over with a great hurry in sneezes energetic it was beginning